In [1]:
import numpy as np

class AABB:
    """轴对齐包围盒"""
    def __init__(self, min_point, max_point):
        self.min_point = np.array(min_point)
        self.max_point = np.array(max_point)
    
    def intersects(self, other):
        """检查是否与另一个AABB相交"""
        return (np.all(self.min_point <= other.max_point) and 
                np.all(other.min_point <= self.max_point))

class OctreeNode:
    """八叉树节点"""
    def __init__(self, bounds, max_objects=4, max_depth=8):
        self.bounds = bounds  # 节点的AABB边界
        self.objects = []     # 存储在该节点的物体
        self.children = []    # 子节点
        self.max_objects = max_objects  # 节点可容纳的最大物体数
        self.max_depth = max_depth      # 最大深度
        self.depth = 0                  # 当前深度
    
    def subdivide(self):
        """将节点分割为8个子节点"""
        if self.depth >= self.max_depth:
            return
        
        center = (self.bounds.min_point + self.bounds.max_point) / 2
        
        # 创建8个子节点
        for i in range(8):
            min_point = np.copy(self.bounds.min_point)
            max_point = np.copy(self.bounds.max_point)
            
            # 根据i的二进制位确定子节点位置
            if i & 1: min_point[0] = center[0]
            else: max_point[0] = center[0]
            if i & 2: min_point[1] = center[1]
            else: max_point[1] = center[1]
            if i & 4: min_point[2] = center[2]
            else: max_point[2] = center[2]
            
            child = OctreeNode(AABB(min_point, max_point), 
                             self.max_objects, self.max_depth)
            child.depth = self.depth + 1
            self.children.append(child)

class Octree:
    """八叉树"""
    def __init__(self, bounds, max_objects=4, max_depth=8):
        self.root = OctreeNode(bounds, max_objects, max_depth)
    
    def insert(self, obj, aabb):
        """插入物体及其AABB"""
        self._insert_recursive(self.root, obj, aabb)
    
    def _insert_recursive(self, node, obj, aabb):
        """递归插入物体"""
        # 如果物体不在节点范围内，返回False
        if not node.bounds.intersects(aabb):
            return False
            
        # 如果没有子节点且未达到对象数量限制，直接插入
        if not node.children and len(node.objects) < node.max_objects:
            node.objects.append((obj, aabb))
            return True
            
        # 如果需要细分且没有子节点
        if not node.children:
            node.subdivide()
            
            # 重新分配当前节点中的物体
            old_objects = node.objects
            node.objects = []
            
            for old_obj, old_aabb in old_objects:
                for child in node.children:
                    if self._insert_recursive(child, old_obj, old_aabb):
                        break
        
        # 尝试将物体插入到子节点
        inserted = False
        for child in node.children:
            if self._insert_recursive(child, obj, aabb):
                inserted = True
                break
                
        # 如果无法插入到任何子节点，存储在当前节点
        if not inserted:
            node.objects.append((obj, aabb))
            
        return True

In [2]:
def find_collisions(self):
    """查找所有可能的碰撞对"""
    collisions = set()
    self._find_collisions_recursive(self.root, collisions)
    return collisions

def _find_collisions_recursive(self, node, collisions):
    """递归查找碰撞"""
    # 检查当前节点中物体之间的碰撞
    for i, (obj1, aabb1) in enumerate(node.objects):
        # 与同一节点中的其他物体检查
        for j, (obj2, aabb2) in enumerate(node.objects[i+1:], i+1):
            if aabb1.intersects(aabb2):
                collisions.add(tuple(sorted((obj1, obj2))))
    
    # 检查子节点中的碰撞
    if node.children:
        for child in node.children:
            self._find_collisions_recursive(child, collisions)

In [5]:
# a) 确定根节点范围
# 首先要为整个碰撞检测系统确定一个初始范围，这就像是为所有参与碰撞检测的物体划定一个 “活动区域”。这个范围是一个能够完全容纳所有待检测物体的三维立方体空间，它构成了八叉树的根节点。

class Octree:
    @staticmethod
    def calculate_scene_bounds(objects):
        """计算场景中所有物体的边界
        
        Args:
            objects: 列表，包含(对象, AABB)元组
            
        Returns:
            AABB: 包含所有物体的边界框
        """
        if not objects:
            raise ValueError("无法计算空场景的边界")
            
        # 初始化边界为第一个物体的AABB
        _, first_aabb = objects[0]
        min_point = np.copy(first_aabb.min_point)
        max_point = np.copy(first_aabb.max_point)
        
        # 遍历所有物体，扩展边界
        for _, aabb in objects[1:]:
            min_point = np.minimum(min_point, aabb.min_point)
            max_point = np.maximum(max_point, aabb.max_point)
            
        # 为边界添加一些余量（比如10%）
        size = max_point - min_point
        margin = size * 0.1
        min_point -= margin
        max_point += margin
        
        return AABB(min_point, max_point)

    @classmethod
    def create_from_objects(cls, objects, max_objects=4, max_depth=8):
        """从物体列表创建八叉树
        
        Args:
            objects: 列表，包含(对象, AABB)元组
            max_objects: 每个节点的最大物体数
            max_depth: 树的最大深度
            
        Returns:
            Octree: 新创建的八叉树
        """
        bounds = cls.calculate_scene_bounds(objects)
        octree = cls(bounds, max_objects, max_depth)
        
        # 将所有物体插入八叉树
        for obj, aabb in objects:
            octree.insert(obj, aabb)
            
        return octree
    
        # 创建一些测试物体
    objects = [
        ("Sphere1", AABB(np.array([0, 0, 0]), np.array([10, 10, 10]))),
        ("Sphere2", AABB(np.array([5, 5, 5]), np.array([15, 15, 15]))),
        ("Cube1", AABB(np.array([-5, -5, -5]), np.array([5, 5, 5]))),
    ]
    
    # 自动计算边界并创建八叉树
    octree = Octree.create_from_objects(objects)
    
    # 查看根节点边界
    print("场景边界:")
    print(f"最小点: {octree.root.bounds.min_point}")
    print(f"最大点: {octree.root.bounds.max_point}")

UFuncTypeError: Cannot cast ufunc 'subtract' output from dtype('float64') to dtype('int32') with casting rule 'same_kind'